# A Simple Example of Properties of IV estimator when Instruments are Weak

In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.stats import gaussian_kde
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
import matplotlib.pyplot as plt

Simulation Design

In [ ]:
np.random.seed(123)

n = 100
beta = 0.1  # 0.1 weak IV
# beta = 1  # 1 strong IV

# One realization
U = norm.rvs(size=n)
Z = norm.rvs(size=n)  # generate instrument
D = beta * Z + U      # generate endogenous variable
Y = D + U             # the true causal effect is 1

# First stage regression
Z1 = sm.add_constant(Z)
model_first_stage = sm.OLS(D, Z1)
results_first_stage = model_first_stage.fit()
print(results_first_stage.summary())  # first stage is very weak here when we set beta = .1

# IV regression
D1 = sm.add_constant(D)
model_iv = IV2SLS(Y, D1, Z1)
results_iv = model_iv.fit()
print(results_iv.summary())

Note that the instrument is weak here (strength of the instrument is controlled by setting $\beta$) -- the t-stat is smaller than any rule-of-thumb suggested in the literature (e.g. $\sqrt{10}$) .

# Run 10000 trials to evaluate distribution of the IV estimator

In [ ]:
np.random.seed(123)

B = 10000  # trials
IVEst = np.zeros(B)

for i in range(B):
    U = norm.rvs(size=n)
    Z = norm.rvs(size=n)  # generate instrument
    D = beta * Z + U  # generate endogenous variable
    Y = D + U  # the true causal effect is 1

    # IV regression
    model_iv = IV2SLS(Y, D, Z)
    results_iv = model_iv.fit()
    IVEst[i] = results_iv.params[0]  # Coefficient of D

# Plot the Actual Distribution against the Normal Approximation (based on Strong Instrument Assumption)

In [ ]:
# Plotting the density of IVEst
plt.figure(figsize=(8, 6))
plt.xlim(-5, 5)
plt.xlabel("IV Estimator - True Effect")
plt.title("Actual Distribution vs Gaussian")

# Plotting density estimate of simulated IV coefficients
val = np.linspace(-5, 5, 200)
kde = gaussian_kde(IVEst - 1, bw_method=.001)  # Need to play with bandwidth depending on problem features.
density = kde(np.linspace(-5, 5, 1000))
plt.plot(np.linspace(-5, 5, 1000), density, color='blue', label='Actual Distribution')

# Plotting Gaussian distribution
var = (1 / (beta ** 2)) * (1 / n)  # theoretical variance of IV
sd = np.sqrt(var)
gaussian = norm.pdf(val, scale=sd)
plt.plot(val, gaussian, color='red', linestyle='--', label='Gaussian Distribution')

plt.legend()

# Calculating rejection frequency
rejection_frequency = np.sum(np.abs(IVEst - 1) / sd > 1.96) / B
print("Rejection Frequency is", rejection_frequency, "while we expect it to be around 0.05")

plt.show()